source: <a href="https://medium.com/@zilliz_learn/graphrag-explained-enhancing-rag-with-knowledge-graphs-3312065f99e1">GraphRAG</a>

RAG: use vector database to retrieve semantically similar text. <br/>
GraphRAG: enhance RAG by incorporating knowledge graphs (KGs). <br/>
Knowledge Graphs (KGs): data structures that store and link related or unrelated data based on their relationships.<br/>

GraphRAG has 2 processes: indexing and querying <br/>

GraphRAG Query <br/>

Global Search: reasoning about holistic questions related to the whole data corpus by leveraging the community summaries.<br/>
Local Search: reasoning about specific entities by fanning out to their neighbors and associated concepts. <br/>

We need to edit this file.
/Users/suphanut_jamonnak/.pyenv/versions/3.11.5/envs/tamu-chatbot/lib/python3.11/site-packages/graphrag/llm/openai/openai_embeddings_llm.py

Let install GraphRAG

In [59]:
! pip install git+https://github.com/zc277584121/graphrag.git
! pip install future
! pip install plotly
! pip install ollama
! pip install nbformat 

  Cloning https://github.com/zc277584121/graphrag.git to /private/var/folders/dg/1lmjvnjn703dyf0qt4nzvyph0000gn/T/pip-req-build-6nfz12rh
  Running command git clone --filter=blob:none --quiet https://github.com/zc277584121/graphrag.git /private/var/folders/dg/1lmjvnjn703dyf0qt4nzvyph0000gn/T/pip-req-build-6nfz12rh
  Resolved https://github.com/zc277584121/graphrag.git to commit c7263931eb9b3b6c7cd72fc11e4a6d4b11e6d48b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

Dependencies

In [2]:
import os
import re
import html
import requests
import networkx as nx
import numpy as np
import plotly.graph_objects as go

Data Preparation

In [48]:
# utility functions
def strip_html(text):
  """ remove HTML tags from a string """
  if not isinstance(text, str):
    return ""
  clean = re.compile("<.*?>")
  return re.sub(clean, "", text)

def preprocess_events(events):
  """ construct dictionary from event data """
  return [
    {
      "title": event["title"],
      "group_title": event["group_title"],
      "url": event["url"],
      "description": strip_html(event["description"]),
      "date": event["date"],
      "date_time": event["date_time"],
      "location": event["location"],
      "location_title": event["location_title"],
      "location_latitude": float(event["location_latitude"]) if event["location_latitude"] != None else 0,
      "location_longitude": float(event["location_longitude"]) if event["location_longitude"] != None else 0,
      "cost": event["cost"],
      "thumbnail": event["thumbnail"],
      "event_types": event["event_types"],
      "event_types_audience": event["event_types_audience"],
    }
    for event in events
  ]

def transform_event_to_sentence(event):
  # extract fields from the event record
  title = event.get("title", None)
  group_title = event.get("group_title", None)
  date = event.get("date", None)
  date_time = event.get("date_time", None)
  location = event.get("location", None)
  description = event.get("description", "").strip()
  location_title = event.get("location_title", None)
  cost = event.get("cost", None)
  event_types = event.get("event_types", None)
  event_types_audience = event.get("event_types_audience", None)
  url = event.get("url", None)

  sentence = ""
  sentence += f"The event titled '{title}' " if title else "The event with no title "
  sentence += f"is organized by {group_title} " if group_title else ""
  sentence += f"and is scheduled to take place on {date}." if date else ""
  sentence += f" At {date_time}." if date_time else ""
  sentence += f" The event will be held at {location}." if location else ""
  sentence += f" ({location_title})." if location_title else ""
  sentence += f" The cost for attending is {cost}." if cost else " The cost for attending is FREE."
  sentence += f" Description: {description}." if description else ""
  sentence += f" This event is categorized under {event_types[0]}." if event_types else ""
  sentence += f" The intended audience for this event is for {','.join(event_types_audience)}." if event_types_audience else ""
  sentence += f" For more details, you can visit the event page at {url}." if url else ""

  return html.unescape(sentence)

In [49]:
# create graphrag data for indexing
index_root = os.path.join(os.getcwd(), 'graphrag_index')
os.makedirs(os.path.join(index_root, 'input'), exist_ok=True)

In [50]:
# read and write tamu events data
file_path = os.path.join(index_root, 'input', 'tamu_events.txt')
tamu_events_url = "https://calendar.tamu.edu/live/json/events/group"
tamu_events = requests.get(tamu_events_url)
data = tamu_events.json()

# transform it to sentences and store in .txt file
preprocessed_data = preprocess_events(data)
with open(file_path, 'w') as f:
  for i, event in enumerate(preprocessed_data):
    sentence = transform_event_to_sentence(event)
    f.write(f"Event {i + 1}: {sentence} \n\n")

Use GraphRAG to index the text file. <br/> 
To initialize your workspace, let’s first run the graphrag.index --init command.

In [51]:
! python -m graphrag.index --init --root ./graphrag_index

Initializing project at ./graphrag_index
⠋ GraphRAG Indexer 

To use OLLAMA (LOCAL LLM) instead of ChatGPT, copy below setting to the graphrag_index

In [52]:
! cp settings.yaml ./graphrag_index/

Run the indexing to create a graph

In [53]:
! python -m graphrag.index --root ./graphrag_index

🚀 Reading settings from graphrag_index/settings.yaml
⠙ GraphRAG Indexer 
⠙ GraphRAG Indexer e.text) - 1 files loaded (0 filtered) ━ 100% … 0…
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (0 filtered) ━ 100% … 0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files l

Run a query, only supports global methods

In [55]:
! python -m graphrag.query --root ./graphrag_index --method global "When is CNN?"



INFO: Reading settings from graphrag_index/settings.yaml
creating llm client with {'api_key': 'REDACTED,len=9', 'type': "openai_chat", 'model': 'mistral', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'request_timeout': 180.0, 'api_base': 'http://localhost:11434/v1', 'api_version': None, 'organization': None, 'proxy': None, 'cognitive_services_endpoint': None, 'deployment_name': None, 'model_supports_json': True, 'tokens_per_minute': 0, 'requests_per_minute': 0, 'max_retries': 10, 'max_retry_wait': 10.0, 'sleep_on_rate_limit_recommendation': True, 'concurrent_requests': 25}

SUCCESS: Global Search Response:  ### Summary of Analyst Reports on Dataset

Convolutional Neural Networks (CNNs) are a significant type of artificial neural network, primarily used for processing data with a grid-like topology such as images [Data: Reports (1, 2, +more)]. They have been applied in various fields including art generation, climate science, and genomics.

Two efficient architectures

Let visualize the graph created by GraphRag

In [3]:
# load the GraphML file
graph = nx.read_graphml("./graphrag_index/output/20241027-191216/artifacts/summarized_graph.graphml")
# create a 3D spring layout with more separation
pos = nx.spring_layout(graph, dim=3, seed=42, k=0.5)
# extract node positions
x_nodes = [pos[node][0] for node in graph.nodes()]
y_nodes = [pos[node][1] for node in graph.nodes()]
z_nodes = [pos[node][2] for node in graph.nodes()]
# extract edge positions
x_edges = []
y_edges = []
z_edges = []

for edge in graph.edges():
    x_edges.extend([pos[edge[0]][0], pos[edge[1]][0], None])
    y_edges.extend([pos[edge[0]][1], pos[edge[1]][1], None])
    z_edges.extend([pos[edge[0]][2], pos[edge[1]][2], None])
# generate node colors based on a colormap
node_colors = [graph.degree(node) for node in graph.nodes()]
node_colors = np.array(node_colors)
node_colors = (node_colors - node_colors.min()) / (node_colors.max() - node_colors.min())  # Normalize to [0, 1]
# create the trace for edges
edge_trace = go.Scatter3d(
    x=x_edges, y=y_edges, z=z_edges,
    mode='lines',
    line=dict(color='lightgray', width=0.5),
    hoverinfo='none'
)
# create the trace for nodes
node_trace = go.Scatter3d(
    x=x_nodes, y=y_nodes, z=z_nodes,
    mode='markers+text',
    marker=dict(
        size=7,
        color=node_colors,
        colorscale='Viridis',  # Use a color scale for the nodes
        colorbar=dict(
            title='Node Degree',
            thickness=10,
            x=1.1,
            tickvals=[0, 1],
            ticktext=['Low', 'High']
        ),
        line=dict(width=1)
    ),
    text=[node for node in graph.nodes()],
    textposition="top center",
    textfont=dict(size=10, color='black'),
    hoverinfo='text'
)

In [4]:
# create the 3D plot
fig = go.Figure(data=[edge_trace, node_trace])
# update layout for better visualization
fig.update_layout(
    title='3D Graph Visualization',
    showlegend=False,
    scene=dict(
        xaxis=dict(showbackground=False),
        yaxis=dict(showbackground=False),
        zaxis=dict(showbackground=False)
    ),
    margin=dict(l=0, r=0, b=0, t=40),
    annotations=[
        dict(
            showarrow=False,
            text="Interactive 3D visualization of GraphML data",
            xref="paper",
            yref="paper",
            x=0,
            y=0
        )
    ]
)
# show the plot
fig.show()